In [472]:
import numpy as np

In [473]:
def puntiCarta(carta):
    valore = carta%10 + 1
    if valore == 1: return 11
    if valore == 3: return 10
    if valore >= 8: return valore - 6
    return 0

class Mazzo:
    def __init__(self):
        self.carte = []
        for carta in range (0,40):
            self.carte.append(carta)
    
    def carteRimaste(self):
        return len(self.carte)
    
    def pesca(self):
        if self.carteRimaste() == 0:
            return None
        if self.carteRimaste() == 1:
            return self.carte.pop()
        n = np.random.randint(0, self.carteRimaste())
        return self.carte.pop(n)

In [474]:
class Briscola:
    def __init__(self):
        self.mazzo = Mazzo()
        self.cartaInFondo = self.mazzo.pesca()
        self.briscola = int(self.cartaInFondo/10)
        self.mano0 = []
        self.mano1 = []
        for _ in range(3):
            self.mano0.append(self.mazzo.pesca())
            self.mano1.append(self.mazzo.pesca())
        self.cartaTirata = self.mano0.pop()

    def fasePescata(self):
        pescata = self.mazzo.pesca()
        self.mano0.append(pescata)
        pescata = self.mazzo.pesca()
        self.mano1.append(pescata)
    
    def cartaVincente(self, carta0, carta1):
        seme0 = int(carta0/10)
        seme1 = int(carta1/10)
        if seme0 == seme1:
            if puntiCarta(carta0) > puntiCarta(carta1): return carta0
            if puntiCarta(carta0) < puntiCarta(carta1): return carta1
            if carta0 > carta1: return carta0
            return carta1
        if seme0 == self.briscola: return carta0
        if seme1 == self.briscola: return carta1
        return carta0
    
    def partitaFinita(self):
        return self.mazzo.carteRimaste() <= 32

In [475]:
import gym
from gym import spaces

class NewEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super(NewEnv, self).__init__()
        self.observation_space = spaces.MultiDiscrete([40, # carte in mano
                                                       40,
                                                       40,
                                                       4, # briscola
                                                       40]) # carta tirata
        self.action_space = spaces.Discrete(3)
        self.reward = 0
        self.briscola = Briscola()
        self.state = (self.briscola.mano1[0],
                      self.briscola.mano1[1],
                      self.briscola.mano1[2],
                      self.briscola.briscola,
                      self.briscola.cartaTirata)

    def step(self, posizioneCarta):
        done = False
        info = {}

        cartaPrimo = self.briscola.cartaTirata
        cartaSecondo = self.briscola.mano1.pop(posizioneCarta)
        cartaVincente = self.briscola.cartaVincente(cartaPrimo, cartaSecondo)
        reward = puntiCarta(cartaPrimo) + puntiCarta(cartaSecondo)
        if cartaSecondo != cartaVincente:
            reward *= 0
        self.briscola.fasePescata()
        self.briscola.cartaTirata = self.briscola.mano0.pop()
        if self.briscola.partitaFinita():
            done = True

        observation = (self.briscola.mano1[0],
                      self.briscola.mano1[1],
                      self.briscola.mano1[2],
                      self.briscola.briscola,
                      self.briscola.cartaTirata)
        
        return observation, reward, done, info
    
    def reset(self):
        self.reward = 0
        self.briscola = Briscola()
        observation = (self.briscola.mano1[0],
                      self.briscola.mano1[1],
                      self.briscola.mano1[2],
                      self.briscola.briscola,
                      self.briscola.cartaTirata)
        return observation
    
    def close (self):
        self.reset()

In [476]:
from sklearn.neural_network import MLPClassifier

#Create environment
env = NewEnv()
n_actions = env.action_space.n

#Create agent
agent = MLPClassifier(hidden_layer_sizes=(25,25))

#initialize agent by feeding it with some random bullshit
agent.fit([env.reset()]*n_actions,range(n_actions))

MLPClassifier(hidden_layer_sizes=(25, 25))

In [477]:
def generate_session():
    states,actions,total_reward = [],[],0
    s = env.reset()    
    while True:
        a = np.random.choice(n_actions,p=agent.predict_proba([s])[0])
        states.append(s)
        actions.append(a)
        s,r,done,_ = env.step(a)
        total_reward+=r
        if done:break
    return states,actions,total_reward

In [478]:
n_samples = 100
percentile = 70 #fit on top 30% (30 best samples)

for i in range(500):
    #sample sessions
    sessions = [generate_session() for _ in range(n_samples)]
    batch_states,batch_actions,batch_rewards = map(np.array,zip(*sessions))
    
    #choose threshold on rewards
    threshold = np.percentile(batch_rewards,percentile)
    elite_states = np.concatenate(batch_states[batch_rewards>=threshold])
    elite_actions = np.concatenate(batch_actions[batch_rewards>=threshold])
    
    #fit our osom neural network >.<
    agent.fit(elite_states,elite_actions)

    print("\repoch %i \tmean reward=%.2f\tthreshold=%.2f"%(i,batch_rewards.mean(),threshold), end="")

/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


epoch 1 	mean reward=1.77	threshold=0.00

/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


epoch 3 	mean reward=1.71	threshold=0.00

/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


epoch 7 	mean reward=1.79	threshold=0.00

/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/friscobuffo/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


epoch 8 	mean reward=2.17	threshold=0.00

ValueError: 'a' and 'p' must have same size

In [ ]:
def printCarta(carta):
    seme = int(carta/10)
    if seme == 0: seme="denara"
    elif seme == 1: seme="bastoni"
    elif seme == 2: seme="spade"
    else: seme="coppe"
    valore = str(carta%10 + 1)
    print(valore + " di " + seme)

def printBriscola(briscola):
    if briscola == 0: print("denara")
    elif briscola == 1: print("bastoni")
    elif briscola == 2: print("spade")
    else: print("coppe")

In [ ]:
def printCarta(carta):
    seme = int(carta/10)
    if seme == 0: seme="denara"
    elif seme == 1: seme="bastoni"
    elif seme == 2: seme="spade"
    else: seme="coppe"
    valore = str(carta%10 + 1)
    print(valore + " di " + seme)

def printBriscola(briscola):
    if briscola == 0: print("denara")
    elif briscola == 1: print("bastoni")
    elif briscola == 2: print("spade")
    else: print("coppe")

In [ ]:
while True:
    briscola = Briscola()
    state = (briscola.mano1[0],
             briscola.mano1[1],
             briscola.mano1[2],
             briscola.briscola,
             briscola.cartaTirata)
    printBriscola(briscola.briscola)
    printCarta(briscola.cartaTirata)
    print()
    printCarta(briscola.mano1[0])
    printCarta(briscola.mano1[1])
    printCarta(briscola.mano1[2])

    mossa = agent.predict([state]*n_actions)
    print(mossa)
    input()

coppe
1 di denara

10 di bastoni
4 di spade
3 di spade
[1 1 1]
bastoni
1 di bastoni

2 di spade
3 di bastoni
6 di denara
[1 1 1]
coppe
6 di denara

5 di denara
8 di denara
3 di bastoni
[1 1 1]
spade
7 di coppe

10 di spade
6 di coppe
5 di coppe
[1 1 1]
bastoni
6 di coppe

10 di coppe
9 di denara
6 di denara
[1 1 1]
bastoni
6 di bastoni

10 di spade
3 di denara
6 di spade
[1 1 1]
spade
9 di spade

3 di coppe
7 di denara
3 di denara
[1 1 1]


KeyboardInterrupt: Interrupted by user